In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


clone repository and move `persian_re` package to use it

In [3]:
!git clone https://github.com/nimaafshar/persian_relation_extraction.git

Cloning into 'persian_relation_extraction'...
remote: Enumerating objects: 203, done.
remote: Counting objects: 100% (193/193), done.
remote: Compressing objects: 100% (138/138), done.
remote: Total 203 (delta 88), reused 155 (delta 50), pack-reused 10
Receiving objects: 100% (203/203), 2.31 MiB | 9.62 MiB/s, done.
Resolving deltas: 100% (88/88), done.


In [4]:
!mv persian_relation_extraction/persian_re/ persian_re/

installing specified requirements for google colab

In [5]:
!pip install -r persian_relation_extraction/requirements_colab.txt

     |████████████████████████████████| 3.8 MB 5.4 MB/s 
     |████████████████████████████████| 316 kB 47.2 MB/s 
     |████████████████████████████████| 636 kB 47.3 MB/s 
     |████████████████████████████████| 6.5 MB 33.9 MB/s 
     |████████████████████████████████| 895 kB 36.8 MB/s 
     |████████████████████████████████| 67 kB 5.4 MB/s 
     |████████████████████████████████| 233 kB 43.4 MB/s 
     |████████████████████████████████| 1.4 MB 31.1 MB/s 
     |████████████████████████████████| 175 kB 45.6 MB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 
     |████████████████████████████████| 235 kB 45.5 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394488 sha256=beb78bdd44d120e97f387df7e4c2c67b52f2b81b9d2c307e3dfca467b6424dd8
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154226 sha256=72300cae56b

checking *CUDA* availablity

In [6]:
!nvidia-smi

Mon Mar 21 20:10:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Entity-Marker tokenizer + Entity Start output model
## Imports

In [7]:
import numpy as np

import torch, gc
import torch.nn as nn
from torch.optim import AdamW

from transformers import BertConfig
from transformers import get_linear_schedule_with_warmup

import collections

from sklearn.metrics import classification_report

In [8]:
from persian_re.preprocess import PerlexData, create_data_loader
from persian_re.tokenizers import BertEntityMarkerTokenizer
from persian_re.settings import Config
from persian_re.models import EntityStartModel
from persian_re.operation import Trainer,TrainingArguments

## GPU configuration

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

device: cuda:0
CUDA is available!  Training on GPU ...


In [10]:
#clear cuda cache
gc.collect()
torch.cuda.empty_cache()

## Load Data

In [11]:
data = PerlexData.get_instance()

## Entity Marker tokenizer

In [12]:
tokenizer: BertEntityMarkerTokenizer = BertEntityMarkerTokenizer.from_pretrained(Config.MODEL_NAME_OR_PATH)

Downloading:   0%|          | 0.00/416k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/134 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/292 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/565 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'BertEntityMarkerTokenizer'.


## Configuration

In [13]:
arguments = TrainingArguments(epochs=20,
                              clip=0.0,
                              train_callback_interval=100)

In [14]:
config = BertConfig.from_pretrained(
    Config.MODEL_NAME_OR_PATH, **{
        'label2id': data.label2ids,
        'id2label': data.id2labels,
        'hidden_dropout_prob': 0.2
    })

In [15]:
config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.2,
  "hidden_size": 768,
  "id2label": {
    "0": "Cause-Effect(e1,e2)",
    "1": "Cause-Effect(e2,e1)",
    "2": "Component-Whole(e1,e2)",
    "3": "Component-Whole(e2,e1)",
    "4": "Content-Container(e1,e2)",
    "5": "Content-Container(e2,e1)",
    "6": "Entity-Destination(e1,e2)",
    "7": "Entity-Origin(e1,e2)",
    "8": "Entity-Origin(e2,e1)",
    "9": "Instrument-Agency(e1,e2)",
    "10": "Instrument-Agency(e2,e1)",
    "11": "Member-Collection(e1,e2)",
    "12": "Member-Collection(e2,e1)",
    "13": "Message-Topic(e1,e2)",
    "14": "Message-Topic(e2,e1)",
    "15": "Other",
    "16": "Product-Producer(e1,e2)",
    "17": "Product-Producer(e2,e1)"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "Cause-Effect(e1,e2)": 0,
    "Cause-

## *DataLoader* objects

In [16]:
train_data_loader = create_data_loader(data.x_train, data.y_train, tokenizer, Config.MAX_LEN,
                                       Config.TRAIN_BATCH_SIZE, data.label2ids)
valid_data_loader = create_data_loader(data.x_valid, data.y_valid, tokenizer, Config.MAX_LEN,
                                       Config.VALID_BATCH_SIZE, data.label2ids)
test_data_loader = create_data_loader(data.x_test,data.y_test, tokenizer, Config.MAX_LEN, Config.TEST_BATCH_SIZE, data.label2ids)

## [CLS] Relation Extraction Model

In [18]:
pt_model = EntityStartModel(config,tokenizer.e1_start_token_id,tokenizer.e2_start_token_id)
pt_model.resize_token_embeddings(len(tokenizer))
pt_model = pt_model.to(device)

Downloading:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of the model checkpoint at HooshvareLab/bert-fa-zwnj-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert.pooler.dense.weight', 'bert.p

## Training
with:
- **AdamW** optimizer with initial learning rate `INITIAL_LEARNING_RATE`
- **Linear Scheduler** with no warmup
- **CrossEntropyLoss** with class weights to balance data

In [19]:
optimizer = AdamW(pt_model.parameters(), lr=Config.INITIAL_LEARNING_RATE)
total_steps = len(train_data_loader) * arguments.epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss(weight=torch.FloatTensor(data.class_weights).to(device))

In [20]:
trainer = Trainer(pt_model,device,loss_fn,optimizer,scheduler,arguments,train_data_loader,valid_data_loader,test_data_loader)

create dir `output` in `BASE_PATH` to save model

In [21]:
!mkdir persian_relation_extraction/output/

In [22]:
trainer.train()

Epochs... :   0%|          | 0/20 [00:00<?, ?it/s]

epoch 1/20


Training... :   0%|          | 0/1000 [00:00<?, ?it/s]

train:  [accuracy:0.5651,f1_weighted:0.5422,loss:1.3656,]


Validation... :   0%|          | 0/90 [00:00<?, ?it/s]

...

train:  [accuracy:0.9867,f1_weighted:0.9867,loss:0.0294,]


Validation... :   0%|          | 0/90 [00:00<?, ?it/s]

validation:  [accuracy:0.7043,f1_weighted:0.7143,loss:1.3519,]
epoch 8/20


Training... :   0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

## Model Evaluation and Comparison

1. using **last epoch** (overfitted) model

In [23]:
y_pred, y_pred_probs, y_true, metrics = trainer.predict_tests()

Predicting tests:   0%|          | 0/250 [00:00<?, ?it/s]

In [24]:
metrics

{'accuracy': 0.7463731865932967, 'f1_weighted': 0.746399584696362}

In [25]:
print(classification_report(y_true, y_pred,target_names=data.labels))

                           precision    recall  f1-score   support

      Cause-Effect(e1,e2)       0.88      0.92      0.90        99
      Cause-Effect(e2,e1)       0.83      0.84      0.84       143
   Component-Whole(e1,e2)       0.76      0.75      0.75       119
   Component-Whole(e2,e1)       0.83      0.65      0.72       110
 Content-Container(e1,e2)       0.80      0.88      0.84       113
 Content-Container(e2,e1)       0.84      0.90      0.87        29
Entity-Destination(e1,e2)       0.85      0.87      0.86       214
     Entity-Origin(e1,e2)       0.77      0.77      0.77       155
     Entity-Origin(e2,e1)       0.94      0.83      0.88        35
 Instrument-Agency(e1,e2)       0.59      0.62      0.61        16
 Instrument-Agency(e2,e1)       0.76      0.72      0.74        99
 Member-Collection(e1,e2)       0.60      0.75      0.67        24
 Member-Collection(e2,e1)       0.79      0.75      0.77       148
     Message-Topic(e1,e2)       0.78      0.88      0.83     

2. using **saved model** (which has the lowest validation loss)

In [26]:
saved_model = torch.load(Config.OUTPUT_PATH)

In [27]:
temp_trainer = Trainer(saved_model,device,loss_fn,optimizer,scheduler,arguments,train_data_loader,valid_data_loader,test_data_loader)

In [28]:
y_pred, y_pred_probs, y_true, metrics = temp_trainer.predict_tests()

Predicting tests:   0%|          | 0/250 [00:00<?, ?it/s]

In [29]:
metrics

{'accuracy': 0.7223611805902952, 'f1_weighted': 0.7186421029665374}

In [30]:
print(classification_report(y_true, y_pred,target_names=data.labels))

                           precision    recall  f1-score   support

      Cause-Effect(e1,e2)       0.81      0.95      0.87        99
      Cause-Effect(e2,e1)       0.80      0.90      0.84       143
   Component-Whole(e1,e2)       0.64      0.81      0.72       119
   Component-Whole(e2,e1)       0.72      0.56      0.63       110
 Content-Container(e1,e2)       0.80      0.84      0.82       113
 Content-Container(e2,e1)       0.79      0.93      0.86        29
Entity-Destination(e1,e2)       0.93      0.75      0.83       214
     Entity-Origin(e1,e2)       0.70      0.85      0.77       155
     Entity-Origin(e2,e1)       0.63      0.83      0.72        35
 Instrument-Agency(e1,e2)       0.38      0.75      0.50        16
 Instrument-Agency(e2,e1)       0.76      0.68      0.72        99
 Member-Collection(e1,e2)       0.44      0.83      0.58        24
 Member-Collection(e2,e1)       0.84      0.71      0.77       148
     Message-Topic(e1,e2)       0.90      0.69      0.78     

## Saving Best Model

In [31]:
torch.save(pt_model,'/content/drive/MyDrive/RE_task/entitymarker_entitystart_v0.1.bin')

## How to prevent overfitting
1. train only last n layers of BERT
2. increase dropout probablity
3. change scheduler (use `ReduceOnPlateu` or non-linear schedulers)